In [1]:
import json

In [7]:
with open("../lhx/piper_real_cnn/data/20250807/seq_140633.json", "r") as f:
    seq = json.load(f)
def find_gripper_transition(seq):
    """
    找到第一个gripper_pos从<0.3变为>0.2的位置
    
    参数:
        seq: 轨迹序列，每个元素是包含'qpos'的字典
    
    返回:
        满足条件的索引，如果找不到返回None
    """
    for i in range(1, len(seq)):
        prev_pos = seq[i-1]['qpos'][-1]  # 前一个gripper位置
        curr_pos = seq[i]['qpos'][-1]    # 当前gripper位置
        
        if prev_pos < 0.03 and curr_pos > 0.02:
            return i  # 返回满足条件的索引
    
    return None  # 如果没有找到

# 使用示例
transition_index = find_gripper_transition(seq)
if transition_index is not None:
    print(seq[transition_index]['ee_pose'])
import cv2
import numpy as np

# 1. 读取视频和设置参数
video_path = "../lhx/piper_real_cnn/data/20250807/seq_140633_pointcloud.mp4"
output_path = "filtered_output_with_gripper.mp4"
center = seq[transition_index]['ee_pose'][:3]  # 替换为实际的center坐标
thresholds = [0.15, 0.1, 0.1]  # x,y,z的阈值范围

# 夹爪框参数（根据用户提供的参数）
# gripper_x = 140
gripper_x = 100

# gripper_y = 160
gripper_width = 480
# gripper_height = 200

# 2. 打开视频流
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception(f"无法打开视频文件: {video_path}")

# 3. 获取视频属性用于输出
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 4. 创建视频写入器
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))



# # 5. 处理每一帧
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 标准化处理 (如示例)
    frame = frame.astype(np.float32) / 255
    frame[..., 0] -= 0.3  # 通道1调整
    frame[..., 1] -= 0.3  # 通道2调整
    
    # 创建掩膜 (三维条件同时满足)
    mask = np.ones_like(frame[..., 0], dtype=bool)
    for c in range(2):  # 对xyz三个通道
        ch = frame[..., c]
        lower = center[c] - thresholds[c]
        upper = center[c] + thresholds[c]
        mask &= (ch >= lower) & (ch <= upper)
    
    # 将夹爪框区域合并到掩膜中
    gripper_mask = np.zeros_like(mask, dtype=bool)
    gripper_mask[:, gripper_x:gripper_x + gripper_width] = True
    mask |= gripper_mask  # 合并掩膜
    
    # 还原标准化调整
    frame[..., 0] = frame[..., 0] + 0.3
    frame[..., 1] = frame[..., 1] + 0.3
    
    # 应用掩膜
    filtered_frame = np.zeros_like(frame)
    filtered_frame[mask] = frame[mask]
    
    # 还原到0-255范围并写入
    out.write((filtered_frame * 255).astype(np.uint8))

# 6. 释放资源
cap.release()
out.release()
print(f"处理完成，结果已保存到: {output_path}")

[0.5110769470783381, -0.021276045287087614, 0.14441210152686698, 0.9983643644900446, 0.0, 0.05717163384397447, 0.0, 1.0]
处理完成，结果已保存到: filtered_output_with_gripper.mp4


[0.5110769470783381, -0.021276045287087614, 0.14441210152686698, 0.9983643644900446, 0.0, 0.05717163384397447, 0.0, 1.0]


处理完成！标注后的视频已保存至: color_with_gripper_box.mp4


In [51]:
print(gripper_x, gripper_y, gripper_width, gripper_height)

140 160 400 200


In [ ]:
gripper_x = 140
gripper_y = 160
gripper_width = 400
gripper_height = 200

In [8]:
gripper_x + gripper_width

580